## Introduction to pandas

[Pandas](http://pandas.pydata.org/) is a Python library for manipulating and analyzing data. In this tutorial, we'll demonstrate key pandas concepts on a real dataset. 

### Concepts

* Reading data
* DataFrames
* Exploring data
* Indexing
* Selection
* Modifying data
* DataFrame functions
* Grouping
* Joining/Merging

We'll be using the Hubway dataset, provided by Boston's bikesharing system.

In [57]:
import pandas as pd
import numpy as np
%matplotlib inline

import psycopg2

In [58]:
# Database connecting params
url = 'toy-db.cex3rfvdw0wv.us-west-2.redshift.amazonaws.com'
database = 'bikeshare'
username = 'read_only'
password = 'U8qz3wmD&Eej*gEB'

conn = psycopg2.connect(
    database=database,
    port=5439,
    password=password,
    user=username,
    host=url
)

### Reading data

Pandas offers a number of [tools](http://pandas.pydata.org/pandas-docs/stable/io.html) for reading and writing data. When working with data from text files, you'll often use the `read_csv` function. Here we'll demonstrate usage of the similar `read_sql` method to get data out of a Redshift database and into a DataFrame.

First, we write our SQL query and store it in a string variable. Here we're selecting 10,000 trips randomly from the trips table.

In [59]:
# Set query
query = """
select * from trips
order by random()
limit 10000
"""

Now we'll retrieve the data from the database, storing the results in a DataFrame variable named `trips`. We pass the `query` and `conn` variables we defined above as parameters to the `read_sql` function.

In [60]:
trips = pd.read_sql(query, conn)

Now let's do the same for stations.

In [61]:
query = """
select * from stations
"""
stations = pd.read_sql(query, conn)

### DataFrames

Pandas provides two [data structures](http://pandas.pydata.org/pandas-docs/dev/dsintro.html) that you'll use for manipulating and analyzing data: the [Series](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.Series.html) and the [DataFrame](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.html). You can think of a Series as a single column of data and a DataFrame as a table of data with one or more columns. Both structures have name properties (column labels) and indexes (row labels). 

Let's take a look at the DataFrame we created above. To see the first 5 rows, we call the DataFrame's `head` method.

In [62]:
trips.head()

,seq_id,trip_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,subsc_type,zip_code,birth_date,gender
0,357153,404493,Closed,1834,2012-07-21 16:38:00,60,2012-07-21 17:09:00,38,B00227,Casual,,0,
1,1441262,1602399,Closed,540,2013-10-17 08:27:00,39,2013-10-17 08:36:00,63,B00089,Casual,,0,
2,1527446,1693855,Closed,180,2013-11-06 18:20:00,74,2013-11-06 18:23:00,87,T01241,Registered,\'02138,0,Male
3,889972,1007616,Closed,420,2013-06-17 08:55:00,68,2013-06-17 09:02:00,91,B00294,Registered,\'02144,0,Female
4,1437267,1598193,Closed,120,2013-10-16 11:50:00,102,2013-10-16 11:52:00,102,B00659,Registered,\'02130,0,Female


We can see that the output looks similar to a data table or spreadsheet, with column labels on the top and numeric row labels running down the left.

To see all the column names and their associated data types, call the DataFrame's `info` method.

In [63]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 13 columns):
seq_id        10000 non-null int64
trip_id       10000 non-null int64
status        10000 non-null object
duration      10000 non-null float64
start_date    10000 non-null datetime64[ns]
strt_statn    10000 non-null int64
end_date      10000 non-null datetime64[ns]
end_statn     10000 non-null int64
bike_nr       10000 non-null object
subsc_type    10000 non-null object
zip_code      10000 non-null object
birth_date    10000 non-null int64
gender        10000 non-null object
dtypes: datetime64[ns](2), float64(1), int64(5), object(5)
memory usage: 1.1+ MB


The output shows us that we have 10,000 rows in the DataFrame. The `read_sql` function automatically converted `start_date` and `end_date` into datetime objects. The `duration` column is encoded as an integer (it's the length of a trip in seconds) while gender is encoded as an object (it's a string with possible values male, female and unknown).

We can use the DataFrame's `describe` method to get summary statistics for all numeric columns.

In [64]:
trips.describe()

,seq_id,trip_id,duration,strt_statn,end_statn,birth_date
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,788471.790700,885336.605700,1122.454800,53.764300,54.021700,461.641300
std,455678.350746,506260.881373,4990.668023,33.543927,33.430911,836.231997
min,95.000000,121.000000,0.000000,3.000000,0.000000,0.000000
25%,392675.250000,444182.750000,404.000000,26.000000,27.000000,0.000000
50%,782420.000000,886882.000000,660.000000,48.000000,48.000000,0.000000
75%,1187856.250000,1331880.750000,1080.000000,74.000000,74.000000,0.000000
max,1578853.000000,1747849.000000,383272.000000,145.000000,145.000000,1995.000000


The three id columns aren't meaningful in this context, but looking at the duration we can see that the median trip duration is 660 seconds and the average is 1,060 seconds. We also get min, max and the confidence interval.

When loading in a new dataset, running `head`, `info` and `describe` is a quick way to get familiar with its structure and content.

### Indexing

Every DataFrame and Series has an [index](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Index.html), similar to row labels in a spreadsheet. Indexes enable quick selection of rows and the ability to perform SQL-style joins on DataFrames. Index values are hashed which yield a constant-time lookup which becomes useful for selection in large DataFrames.

To create an index on an existing DataFrame, use the `set_index` method passing in the desired column as a parameter. We'll turn our attention to the `stations` DataFrame. We have a `station_id` column which unqiuely identifies the station, and our `trips` DataFrame has multiple station id columns we can utlizize for joining the two DataFrames together.

Here we set the index of `stations` to `station_id` and reassign the resulting DataFrame back into `stations`.

In [65]:
stations = stations.set_index('statn_id')

Now when we call the DataFrame's `info` method, we no longer see `station_id` listed as a column. 

In [66]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 143 to 142
Data columns (total 6 columns):
terminal     142 non-null object
station      142 non-null object
municipal    142 non-null object
lat          142 non-null float64
lng          142 non-null float64
status       142 non-null object
dtypes: float64(2), object(4)
memory usage: 7.8+ KB


When we call the `head` method, we now see `station_id` on the far left side of the output with values bolded.

In [67]:
stations.head()

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,S32012,Summer St at Cutter St,Somerville,42,-71,Existing
7,A32000,Fan Pier,Boston,42,-71,Existing
12,B32002,Ruggles Station / Columbus Ave.,Boston,42,-71,Existing
14,B32003,HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.,Boston,42,-71,Existing


### Selection

Pandas offers a lot of flexibility for selecting subsets of data from DataFrames. For example, we have at least three ways of selecting a column. 

We can use a dictionary-style notation where we pass in the column name as a string. Here we'll select the head of the station column. 

In [68]:
stations['station'].head()

statn_id
143    Somerville Hospital at Highland Ave / Crocker St
145                              Summer St at Cutter St
7                                              Fan Pier
12                      Ruggles Station / Columbus Ave.
14     HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.
Name: station, dtype: object

We can also use an object property-style notation where we append the name of the column to the DataFrame variable separated by a period character.

In [69]:
stations.station.head()

statn_id
143    Somerville Hospital at Highland Ave / Crocker St
145                              Summer St at Cutter St
7                                              Fan Pier
12                      Ruggles Station / Columbus Ave.
14     HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.
Name: station, dtype: object

Finally, we can also select based on the numeric position of the column. From calling `head` above, we know that `station` is the second column in the DataFrame. We can access it using the `iloc` property, which allows us to select ranges of rows and columns using integers, with a similar syntax to selecting elements from Python lists.

Let's select the station column and all its corresponding rows.

In [70]:
stations.iloc[:,1]

statn_id
143     Somerville Hospital at Highland Ave / Crocker St
145                               Summer St at Cutter St
7                                               Fan Pier
12                       Ruggles Station / Columbus Ave.
14      HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.
19                                          Buswell Park
25                              Tremont St / W Newton St
31                                         Seaport Hotel
33                                 Kenmore Sq / Comm Ave
38                               TD Garden - Legends Way
40                           Lewis Wharf - Atlantic Ave.
45                            Yawkey Way at Boylston St.
50                             Boylston St / Berkeley St
52                              Newbury St / Hereford St
57                           Columbus Ave. at Mass. Ave.
59       Chinatown Gate Plaza - Surface Rd. at Beach St.
65                 Boston Convention & Exhibition Center
69               Cooli

We enclose in brackets the numeric indexes of the rows we wish to select followed by the columns separated by a comma. In this case, we specified all rows by using the colon unbounded by ab integer on either side, followed by a `1` after the comma to select the second column (DataFrames are 0-indexed like all Python data structures).

Here's what we would do if we wanted the first 5 rows including all columns.

In [71]:
stations.iloc[:5]

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,S32012,Summer St at Cutter St,Somerville,42,-71,Existing
7,A32000,Fan Pier,Boston,42,-71,Existing
12,B32002,Ruggles Station / Columbus Ave.,Boston,42,-71,Existing
14,B32003,HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.,Boston,42,-71,Existing


Now let's get the first 5 rows and the second and third columns. We'll pass in a list to `iloc` of the columns we want to select.

In [72]:
cols = [1,2]
stations.iloc[:5,cols]

,station,municipal
statn_id,,
143,Somerville Hospital at Highland Ave / Crocker St,Somerville
145,Summer St at Cutter St,Somerville
7,Fan Pier,Boston
12,Ruggles Station / Columbus Ave.,Boston
14,HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.,Boston


If we want to select from the DataFrame based on a value in the index (by row label essentially), we use the `loc` property. In `stations` our `station_id` is stored as a string so we supply a string to `loc`.

In [73]:
stations.loc[39]

terminal                            C32006
station      Washington St. at Rutland St.
municipal                           Boston
lat                                     42
lng                                    -71
status                            Existing
Name: 39, dtype: object

#### Selection Via Boolean Indexing

Those are the basics for selecting data by numeric position or label. Oftentimes we want to select data based on some criteria. In pandas boolean indexing is the primary method for performing this type of selection.

What if we wanted to select all rows and columns for stations located in Brookline? Here's what that looks like.

In [74]:
stations[stations['municipal'] == 'Brookline']

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
69,K32001,Coolidge Corner - Beacon St @ Centre St,Brookline,42,-71,Existing
86,K32003,Brookline Village - Station Street @ MBTA,Brookline,42,-71,Existing
127,K32004,JFK Crossing at Harvard St. / Thorndike St.,Brookline,42,-71,Existing
82,K32002,Brookline Town Hall / Library Washington St,Brookline,42,-71,Removed
126,K32002,Washington Square at Washington St. / Beacon St.,Brookline,42,-71,Existing


Let's break this into pieces so we can see what's happening under the hood. You'll see within the enclosing brackets, we have `stations['municipality'] == 'Brookline'`. This is what we get when execute that statement.

In [75]:
stations['municipal'] == 'Brookline'

statn_id
143    False
145    False
7      False
12     False
14     False
19     False
25     False
31     False
33     False
38     False
40     False
45     False
50     False
52     False
57     False
59     False
65     False
69      True
71     False
76     False
81     False
86      True
88     False
92     False
97     False
103    False
108    False
110    False
115    False
120    False
       ...  
27     False
34     False
36     False
42     False
47     False
49     False
55     False
60     False
62     False
67     False
72     False
74     False
78     False
83     False
85     False
90     False
95     False
99     False
101    False
105    False
107    False
112    False
117    False
119    False
124    False
129    False
131    False
135    False
140    False
142    False
Name: municipal, dtype: bool

Notice that it's a `Series` of booleans sharing the same index as the `stations` DataFrame. The result of that code was a comparison of every value in the `municipality` column to the string `'Brookline'` with a boolean result.

Just to make sure, let's check station 82 which is showing a value of `True` above.

In [76]:
stations.loc[82]

terminal                                          K32002
station      Brookline Town Hall / Library Washington St
municipal                                      Brookline
lat                                                   42
lng                                                  -71
status                                           Removed
Name: 82, dtype: object

We see that this station is indeed located in Brookline.

We can also separate the selection of stations in Brookline into two steps: one for the creation of the boolean series and one for the selections from `stations`. When you pass a DataFrame a boolean series, pandas broadcasts that series across all rows where the corresponding index has evaluated to `True`.

In [77]:
# Create boolean series
brookline_select = stations.municipal == 'Brookline'
# Return rows from stations
stations[brookline_select]

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
69,K32001,Coolidge Corner - Beacon St @ Centre St,Brookline,42,-71,Existing
86,K32003,Brookline Village - Station Street @ MBTA,Brookline,42,-71,Existing
127,K32004,JFK Crossing at Harvard St. / Thorndike St.,Brookline,42,-71,Existing
82,K32002,Brookline Town Hall / Library Washington St,Brookline,42,-71,Removed
126,K32002,Washington Square at Washington St. / Beacon St.,Brookline,42,-71,Existing


We can also negate a boolean series using the `~` operator.

In [78]:
# Boolean series for all stations in Boston
boston_select = stations.municipal == 'Boston'
# Use ~ to negate the boolean values
stations[~boston_select].head()

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,S32012,Summer St at Cutter St,Somerville,42,-71,Existing
69,K32001,Coolidge Corner - Beacon St @ Centre St,Brookline,42,-71,Existing
71,S32004,Conway Park - Somerville Avenue,Somerville,42,-71,Existing
76,M32012,Central Sq Post Office / Cambridge City Hall a...,Cambridge,42,-71,Existing


A scroll through the resulting DataFrame shows that we don't have any rows where the munipality equals Boston.

We can also combine multiple boolean indexes using the `&` (logical and) and `|` (logical or) operators. Let's select stations in Somerville and Cambridge using the `or` operator, wrapping each boolean expression in parentheses.

In [79]:
stations[(stations.municipal == 'Somerville') | (stations.municipal == 'Cambridge')].head()

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,S32012,Summer St at Cutter St,Somerville,42,-71,Existing
71,S32004,Conway Park - Somerville Avenue,Somerville,42,-71,Existing
76,M32012,Central Sq Post Office / Cambridge City Hall a...,Cambridge,42,-71,Existing
88,M32010,Inman Square at Vellucci Plaza / Hampshire St,Cambridge,42,-71,Existing


We could perform that same selection using the `.isin` method. This method evaluates to a boolean series by checking for membership in the provided list.

In [80]:
# Create a list of municipalities we want to select
muni_list = ['Cambridge', 'Somerville']
# Pass list to isin method
stations[stations.municipal.isin(muni_list)].head()

,terminal,station,municipal,lat,lng,status
statn_id,,,,,,
143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,S32012,Summer St at Cutter St,Somerville,42,-71,Existing
71,S32004,Conway Park - Somerville Avenue,Somerville,42,-71,Existing
76,M32012,Central Sq Post Office / Cambridge City Hall a...,Cambridge,42,-71,Existing
88,M32010,Inman Square at Vellucci Plaza / Hampshire St,Cambridge,42,-71,Existing


### Modifying data

We can use our selection tools to modify the contents of a DataFrame and add new values. 

The `terminal` column isn't really interesting for our analysis, we can remove it from the DataFrame using the `drop` method. We pass in the column we want to drop and the axis on which to look for that column (0 for row, 1 for column).

In [81]:
# Drop the terminal column
stations = stations.drop('terminal', axis=1)
# Call info to see results
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 143 to 142
Data columns (total 5 columns):
station      142 non-null object
municipal    142 non-null object
lat          142 non-null float64
lng          142 non-null float64
status       142 non-null object
dtypes: float64(2), object(3)
memory usage: 6.7+ KB


The output of info shows no terminal column.

If we wanted to add a new column to the DataFrame based on existing data, we can store the results of a numeric operation under a new label. Here we add a new column called `duration_hours` which is the trip duration expressed in hours.

In [82]:
# Similar to selection, input new column name in brackets with assignment
trips['duration_hours'] = trips.duration / 3600
# Take a look at the results
trips[['duration','duration_hours']].head()

,duration,duration_hours
0,1834,0.509444
1,540,0.150000
2,180,0.050000
3,420,0.116667
4,120,0.033333


Another way to add a new column based on existing data is using the `apply` method. Let's say we were interested in the day of week that a ride occurred on. We can apply a day of week function to the trip's start time and store the result in a new column. We use an anonymous function with a Python [lambda expression](https://docs.python.org/2/reference/expressions.html#lambda).

In [83]:
# Apply the lambda function to the start_date column
trips['start_day_of_week'] = trips.start_date.apply(lambda x: x.dayofweek)
# Take a look at the results
trips[['start_date','start_day_of_week']].head()

,start_date,start_day_of_week
0,2012-07-21 16:38:00,5
1,2013-10-17 08:27:00,3
2,2013-11-06 18:20:00,2
3,2013-06-17 08:55:00,0
4,2013-10-16 11:50:00,2


We can add new rows to a DataFrame using the `append` method. Here we'll append a the last 5 rows of the `stations` DataFrame to a copy of itself.

In [84]:
stations_copy = stations.copy()
stations_copy = stations_copy.append(stations_copy.iloc[-5:])

Now when we examine the lengths of the two DataFrames we see that the copy with append is five rows longer.

In [85]:
print 'Length of stations df: {0}'.format(len(stations))
print 'Length of stations appended df: {0}'.format(len(stations_copy))

Length of stations df: 142
Length of stations appended df: 147


### DataFrame Functions

DataFrames supply plenty of functionality for aggregating data and performing other numerical computation. Below we're calling the `count` method on the `trips` DataFrame. We see that it performs the count on all columns in the DataFrame regardless of type and returns a Series indexed by the column labels.

In [86]:
trips.count()

seq_id               10000
trip_id              10000
status               10000
duration             10000
start_date           10000
strt_statn           10000
end_date             10000
end_statn            10000
bike_nr              10000
subsc_type           10000
zip_code             10000
birth_date           10000
gender               10000
duration_hours       10000
start_day_of_week    10000
dtype: int64

Because aggregation functions return a Series, we can use the column label selection we learned in the previous selection. Here we select the count of `zip_code` from `trips`.

In [87]:
trips.count()['zip_code']

10000

We also have other useful functions including `sum`. We run `sum` in the cell below and see that it returns results for numeric columns only.

In [88]:
trips.sum()

seq_id               7.884718e+09
trip_id              8.853366e+09
duration             1.122455e+07
strt_statn           5.376430e+05
end_statn            5.402170e+05
birth_date           4.616413e+06
duration_hours       3.117930e+03
start_day_of_week    2.909900e+04
dtype: float64

Summing id columns is meaningless, we can exclude columns by specifically specifying the columns we wish to aggregate using column label selection. Let's find the average trip duration using the `mean` method.

In [89]:
trips['duration'].mean()

1122.4548

You can read about all the functions available [here](http://pandas.pydata.org/pandas-docs/stable/basics.html#descriptive-statistics).

In addition to descriptive statistics such as `sum` and `mean`, pandas provides functions for counting frequencies and extracting unique values. To get value counts for a categorical column, use the `value_counts` method. Here we find the count of stations by municipality.

In [90]:
stations.municipal.value_counts()

Boston        97
Cambridge     28
Somerville    12
Brookline      5
dtype: int64

To get all unique values we use `unique`.

In [91]:
stations.municipal.unique()

array(['Somerville', 'Boston', 'Brookline', 'Cambridge'], dtype=object)

Finally, to get the number of unqiue values, we use `nunique`.

In [92]:
stations.municipal.nunique()

4

### Grouping

In the previous section we applied functions to the contents of the entire DataFrame. In many cases, you'll want to apply these functions to groups of data. Think about writing a SQL query to find the average order size for a customer of an ecommerce company. You would probably write something like `select customer_id, avg(amount) from order group by 1`. In pandas, we can perform similar opertions using `group_by` objects.

Previously, we found the average trip duration for all trips in the DataFrame. What if we wanted to see average trip duration by subscription type? Here's how we can do that using `group_by`.

In [93]:
trips.groupby('subsc_type')['duration'].mean()

subsc_type
Casual        2190.538488
Registered     695.041865
Name: duration, dtype: float64

The result of calling `mean` on the `group_by` is a series indexed by the grouping field. We can group on multiple fields which yields a multi-level index. Here we'll find median trip durations by starting station id and gender by passing in a list of fields to the `group_by` call.

In [94]:
trips.groupby(['strt_statn','gender'])['duration'].median().head()

strt_statn  gender
3                     1197.5
            Female     947.0
            Male       660.0
4                     1153.5
            Female     431.0
Name: duration, dtype: float64

### Joining/Merging

DataFrames can be [merged and joined](http://pandas.pydata.org/pandas-docs/stable/merging.html) using a SQL-style syntax. The `join` method will attempt to join two DataFrames by matching on their indexes. As in SQL, you can specify the type of join (inner, left, right).

In the `trips` DataFrame we have a station id representing the station from which a trip started but we do not have the station's name. We can get station name from the `stations` DataFrame and combine the two with join.

First, let's calculate the total number of trips by station id from the `trips` DataFrame.

In [95]:
# Count of trip id's for each station
trip_station_totals = trips.groupby('strt_statn')[['trip_id']].count()
# rename column to something more descriptive
trip_station_totals.columns = ['trip_count']

Now if we examine the DataFrame we see that the index is `start_station_id` and we have a column called `trip_count` which contains the total number of trips originating from that station.

In [96]:
trip_station_totals.head()

,trip_count
strt_statn,
3,57
4,121
5,78
6,137
7,60


For the join to `stations`, we'll need to set an index on station id and select the station name column. The resulting DataFrame contains a single column of station names and is indexed by station id.

In [97]:
# Set index and select column
station_names = stations
# Turn the index into a numeric type for matching
station_names.index = station_names.index.astype('int')
station_names.head()

,station,municipal,lat,lng,status
statn_id,,,,,
143,Somerville Hospital at Highland Ave / Crocker St,Somerville,42,-71,Existing
145,Summer St at Cutter St,Somerville,42,-71,Existing
7,Fan Pier,Boston,42,-71,Existing
12,Ruggles Station / Columbus Ave.,Boston,42,-71,Existing
14,HMS / HSPH - Ave. Louis Pasteur at Longwood Ave.,Boston,42,-71,Existing


Now that we have two DataFrames with matching indexes, we can perform the join using the `join` method. By default it will perform an inner join.

In [98]:
# Perform the join
trip_totals_w_station_name = trip_station_totals.join(station_names)
# Look at the results
trip_totals_w_station_name.head()

,trip_count,station,municipal,lat,lng,status
strt_statn,,,,,,
3,57,Colleges of the Fenway,Boston,42,-71,Existing
4,121,Tremont St. at Berkeley St.,Boston,42,-71,Existing
5,78,Northeastern U / North Parking Lot,Boston,42,-71,Existing
6,137,Cambridge St. at Joy St.,Boston,42,-71,Existing
7,60,Fan Pier,Boston,42,-71,Existing


The resulting DataFrame is indexed by station id and contains the columns `trip_count` and `station` from the two DataFrames we joined.